In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [22]:
import numpy as np
import pandas as pd
# update 순서

# # 정확도 0.7013221153846154
df = pd.read_csv('./kaggle/input/spaceship-titanic/train_Destination_update.csv')

# score 0.8372415742369386
# df = pd.read_csv('./kaggle/input/spaceship-titanic/train_VRDeck_update.csv')

df_train = df

,PassengerId,HomePlanet,CryoSleep,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,...,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,2.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0.0
1,0002_01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,2.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1.0
2,0003_01,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0.0
3,0003_02,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0.0
4,0004_01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,2.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8489,8574_02,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,2.0,53.0,0.0,1042.0,0.0,2.0,1.0,11.0,Yakers Welte,0.0
8490,4351_01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,2.0,21.0,0.0,641.0,0.0,409.0,0.0,0.0,Timmy Carezquez,0.0
8491,5070_01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,2.0,20.0,0.0,542.0,175.0,0.0,5.0,0.0,Lillia Collinson,1.0
8492,8602_03,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,2.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,Elson Weissey,0.0


In [23]:
df_train.isnull().sum()

PassengerId       0
HomePlanet        0
CryoSleep         0
Cabin_A           0
Cabin_B           0
Cabin_C           0
Cabin_D           0
Cabin_E           0
Cabin_F           0
Cabin_G           0
Cabin_T           0
Cabin_num         0
Cabin_port        0
Destination       0
Age             175
VIP               0
RoomService     158
FoodCourt       166
ShoppingMall    186
Spa             167
VRDeck          160
Name            198
Transported       0
dtype: int64

In [24]:
df_train.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin_A', 'Cabin_B',
       'Cabin_C', 'Cabin_D', 'Cabin_E', 'Cabin_F', 'Cabin_G', 'Cabin_T',
       'Cabin_num', 'Cabin_port', 'Destination', 'Age', 'VIP', 'RoomService',
       'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name', 'Transported'],
      dtype='object')

In [26]:
# train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBRegressor


def fillnull(column_name):

    global X_train, X_test, y_train, y_test

    # df_FoodCourt =  df_train중에서 FoodCourt null값이 존재하는 열만 추출 
    # -> 훈련된 모델의 값으로 다시 채울 예정
    df_column_name = df_train[df_train[column_name].isnull()]
    #df_train 중에서 FoodCourt 의 null값이 존재하는 열만 추출 -> 추후 훈련할 데이터프레임
    df_train01 = df_train.dropna(subset=[column_name])
    # df_train
    df_train[df_train[column_name].isnull()]

    X_train, X_test, y_train, y_test = train_test_split(df_train01.drop(['PassengerId',column_name,'Name'],axis=1),df_train01[column_name], test_size=0.2, random_state=42)

    # scaler
    scaler = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    # 모델
    model = xgb.XGBRegressor(early_stopping_rounds=10)
    eval_set = [(X_test, y_test)]
    model.fit(X_train, y_train, eval_set=eval_set, verbose=False)#, verbose=True

    pred = model.predict(X_test)
    score = model.score(X_train,y_train)
    print(score)

    global df_update
    # df_01 은 df_train 중에서 FoodCourt 의 값이 null인 경우의 데이터프레임
    df_column_name_null = df_column_name.drop(column_name,axis=1)
    # 훈련된 xgboost모델을 사용해서 df_homeplanet_null의 HomePlanet값을 예측
    pred = model.predict(df_column_name_null.drop(['PassengerId','Name'],axis=1))
    pred.max()
    pred = np.round(pred)

    # 예측된 값을 다시 df_homeplanet으로 삽입(null값 채우기)
    df_column_name[column_name] = pred
    df_column_name
    
    # null값을 채운 dataframe과 df_train01 합치기 
    df_update = pd.concat([df_train01,df_column_name],axis=0)

fillnull('VRDeck')


0.5928344657888602


C:\Users\deepBlue\AppData\Local\Temp\ipykernel_8944\1698436787.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_column_name[column_name] = pred


In [19]:
df_update[df_update['VRDeck'].isnull()]

,PassengerId,HomePlanet,CryoSleep,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,...,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported


In [20]:
df_update.to_csv('./kaggle/input/spaceship-titanic/train_VRDeck_update.csv', index=False)